# Azure AI Agent with Explicit Settings Example

This notebook demonstrates creating Azure AI Agents with explicit configuration settings rather than relying on environment variable defaults.

## Features Covered:
- Explicit configuration of Azure AI client settings
- Direct specification of project endpoint and model deployment
- Custom agent naming during client creation
- Alternative to environment variable-based configuration

## Prerequisites

Before running this notebook, ensure you have:
- Azure CLI installed and authenticated (`az login --use-device-code`)
- Access to an Azure AI Foundry project with deployed models
- Environment variables set up (see Initial Setup section below)

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [ ]:
import asyncio
import os
from random import randint
from typing import Annotated
import os
from pathlib import Path

from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity import AzureCliCredential
from pydantic import Field
from dotenv import load_dotenv  # For loading environment variables from .env file

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../../.env')  # Load environment variables from .env file

# Get connection string and model deployment for explicit settings
conn_string = os.environ.get("PROJECT_CONNECTION_STRING")
model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

print(f"🔗 Using Connection String: {conn_string[:50]}..." if conn_string else "❌ No connection string found")
print(f"🤖 Using Model: {model_deployment}")
print("✅ Environment variables loaded for explicit settings configuration")

In [ ]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

## Validate Explicit Settings

Before creating agents, let's validate our explicit configuration settings:

In [ ]:
# Validate explicit settings before creating agents
print("🔍 Validating Explicit Settings Configuration")
print("=" * 50)

# Check PROJECT_CONNECTION_STRING
if conn_string:
    print(f"✅ PROJECT_CONNECTION_STRING: {conn_string[:50]}...")
    endpoint_configured = True
else:
    print("❌ PROJECT_CONNECTION_STRING: Not configured")
    endpoint_configured = False

# Check MODEL_DEPLOYMENT_NAME  
if model_deployment:
    print(f"✅ MODEL_DEPLOYMENT_NAME: {model_deployment}")
    model_configured = True
else:
    print("❌ MODEL_DEPLOYMENT_NAME: Not configured")
    model_configured = False

# Overall validation
if endpoint_configured and model_configured:
    print("\n🎉 All explicit settings are properly configured!")
    print("✅ Ready to create agents with explicit configuration")
else:
    print("\n⚠️  Some explicit settings are missing")
    print("💡 Please check your environment variables")

print("=" * 50)

In [ ]:
async def main() -> None:
    print("=== Azure AI Chat Client with Explicit Settings ===")

    # Use explicit settings instead of environment variable defaults
    # This approach provides more control over configuration
    credential = AzureCliCredential()
    
    async with AzureAIAgentClient(
        project_endpoint=conn_string,  # Explicitly set project endpoint
        model_deployment_name=model_deployment,  # Explicitly set model deployment
        async_credential=credential,  # Explicitly set credential
    ) as chat_client:
        
        # Create agent with explicit name and settings
        agent = chat_client.create_agent(
            name="ExplicitWeatherAgent",  # Explicit agent name
            instructions="You are a helpful weather agent with explicitly configured settings.",
            tools=get_weather,
        )
        
        print(f"Created agent with explicit settings: ExplicitWeatherAgent")
        print(f"Project Endpoint: {conn_string[:50]}...")
        print(f"Model Deployment: {model_deployment}")

        try:
            # Test with multiple queries to demonstrate explicit configuration
            queries = [
                "What's the weather like in New York?",
                "How about in Paris?",
                "Compare the weather between these two cities"
            ]
            
            for i, query in enumerate(queries, 1):
                print(f"\n--- Query {i} ---")
                print(f"🤔 User: {query}")
                result = await agent.run(query)
                print(f"🤖 Agent: {result}")
                
        except Exception as e:
            print(f"❌ Error with explicit settings: {str(e)}")
            
        print(f"\n💡 Explicit settings configuration completed successfully")

# Run the main function
await main()

## Alternative Explicit Settings Patterns

This example shows different ways to explicitly configure Azure AI agents:

In [ ]:
async def alternative_explicit_patterns() -> None:
    """Demonstrate different explicit configuration patterns."""
    print("=== Alternative Explicit Settings Patterns ===")
    
    # Pattern 1: Hardcoded explicit settings (for testing/development)
    print("\n🔧 Pattern 1: Hardcoded Explicit Settings")
    print("(Useful for development and testing)")
    
    credential = AzureCliCredential()
    
    async with AzureAIAgentClient(
        project_endpoint=conn_string,  # From environment but explicitly passed
        model_deployment_name="gpt-4o",  # Hardcoded explicit value
        async_credential=credential,
    ) as chat_client:
        
        agent = chat_client.create_agent(
            name="HardcodedSettingsAgent",
            instructions="I am configured with hardcoded explicit settings.",
            tools=get_weather,
        )
        
        print(f"✅ Created agent with hardcoded settings")
        result = await agent.run("What's the weather in Tokyo with hardcoded settings?")
        print(f"🤖 Result: {result}")
    
    # Pattern 2: Dictionary-based configuration 
    print(f"\n🔧 Pattern 2: Dictionary-based Configuration")
    print("(Useful for configuration management)")
    
    # Configuration dictionary
    azure_config = {
        "project_endpoint": conn_string,
        "model_deployment_name": model_deployment,
        "agent_name": "ConfigDrivenAgent",
        "instructions": "I am configured through a configuration dictionary."
    }
    
    async with AzureAIAgentClient(
        project_endpoint=azure_config["project_endpoint"],
        model_deployment_name=azure_config["model_deployment_name"],
        async_credential=credential,
    ) as chat_client:
        
        agent = chat_client.create_agent(
            name=azure_config["agent_name"],
            instructions=azure_config["instructions"],
            tools=get_weather,
        )
        
        print(f"✅ Created agent with config dictionary")
        result = await agent.run("What's the weather in London with config-driven settings?")
        print(f"🤖 Result: {result}")
    
    print(f"\n💡 Alternative explicit patterns completed successfully")

# Run the alternative patterns example
await alternative_explicit_patterns()

In [ ]:
async def explicit_settings_comparison() -> None:
    """Compare explicit settings vs environment variable approaches."""
    print("=== Explicit Settings vs Environment Variables ===")
    
    credential = AzureCliCredential()
    
    # Environment variable approach (for comparison)
    print("\n📝 Environment Variable Approach:")
    print("- Uses default values from environment")
    print("- Less explicit but more flexible for deployment")
    print("- Configuration changes require env file updates")
    
    # Explicit settings approach 
    print(f"\n🎯 Explicit Settings Approach:")
    print("- All configuration values specified in code")
    print("- More explicit and visible")
    print("- Configuration changes require code updates")
    print("- Better for testing and development scenarios")
    
    # Demonstrate explicit parameter passing
    async with AzureAIAgentClient(
        project_endpoint=conn_string,  # Explicitly passed
        model_deployment_name=model_deployment,  # Explicitly passed  
        async_credential=credential,  # Explicitly passed
    ) as chat_client:
        
        agent = chat_client.create_agent(
            name="ExplicitComparisonAgent",  # Explicitly set
            instructions="I demonstrate explicit vs implicit configuration.",  # Explicitly set
            tools=get_weather,  # Explicitly passed
        )
        
        print(f"\n✅ Agent created with all explicit settings")
        result = await agent.run("Compare explicit vs environment variable approaches in weather for Paris")
        print(f"🤖 Agent Response: {result}")
    
    print(f"\n✨ Explicit settings provide better control and visibility")

# Run the comparison example
await explicit_settings_comparison()

## Summary

This notebook demonstrates how to use **explicit settings** instead of relying on environment variable defaults when working with Azure AI Agents.

### Key Takeaways:

✅ **Explicit Configuration**: All settings are explicitly passed to the `AzureAIAgentClient`
- `project_endpoint`: Explicitly specify the Azure AI project endpoint
- `model_deployment_name`: Explicitly set the model deployment name
- `async_credential`: Explicitly pass the authentication credential

✅ **Benefits of Explicit Settings**:
- Better code readability and documentation
- Easier testing and development
- More predictable behavior
- Configuration is visible in the code

✅ **Configuration Patterns**:
- Hardcoded values for development/testing
- Dictionary-based configuration management
- Explicit parameter passing vs environment variables

✅ **Best Practices**:
- Use explicit settings for development and testing scenarios
- Use environment variables for production deployments
- Choose the approach that best fits your use case and deployment strategy

The explicit settings approach provides maximum control and visibility over your Azure AI Agent configuration.